In [27]:
#%pip install pandas
#%pip install numpy
import pandas as pd
import numpy as np
import json

In [28]:
# Read the CSV file
# The CSV file is saved in the same directory as this notebook
csv_data = pd.read_csv('GameFact_MockData.xlsx - Sheet1.csv')
csv_data.head()

# Read the JSON file
# The JSON file is saved in the same directory as this notebook
## json_data = pd.read_json('GameFactMockData2.json')

with open('GameFactMockData2.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)
json_data = pd.json_normalize(json_data, record_path=["gamefact_data"])

json_data.head()

# Combine the CSV and JSON data into a single DataFrame
raw_data = pd.concat([csv_data, json_data], axis=0, ignore_index=True)
raw_data.head()
raw_data.shape


(35, 15)

In [29]:
cleaned_data_1 = raw_data.dropna(subset=['EventID', 'StadiumID', 'RefereeID', 'GameID', 'TimeID', 'DateID', 'GameDuration', 'GameTeam', 'GameInterruption', 'GameResult'])
cleaned_data_1.shape

(30, 15)

In [31]:
expected_types = {
    'EventID': int,
    'StadiumID': int,
    'RefereeID': int,
    'GameID': int,
    'TimeID': int,
    'DateID': int,
    'GameDuration': int,
    'GameTeam': str,
    'GameInterruption': int,
    'GameResult': str
}

def is_valid_row(row):
    for col, dtype in expected_types.items():
        try:
            # Convert value to expected type
            value = dtype(row[col])
            # Reject negative values for integer columns
            if dtype is int and value < 0:
                return False
        except (ValueError, TypeError):
            return False  # Reject the row if conversion fails
    return True

# Apply row-wise filtering
cleaned_data_2 = cleaned_data_1[cleaned_data_1.apply(is_valid_row, axis=1)].reset_index(drop=True)